In [9]:
# Import numpy for random number generation
import numpy as np
from random import randbytes
# importing Qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler
from qiskit_aer import AerSimulator
# Import basic plotting tools
from helpers import *
from qiskit.visualization import plot_histogram
from dotenv import load_dotenv
import os

load_dotenv()
provider = QiskitRuntimeService(token=os.environ["ibm_token"], channel="ibm_quantum")

# Selecting a backend
real_backend = provider.backend("ibm_brisbane")
backend = AerSimulator().from_backend(real_backend)

In [10]:
key_len = 16  # for a local backend n can go as up as 23, after that it raises a Memory Error
runs = 10 # Number of unique key exchanges
shots = 2 # Number of times each circuit is executed
alices = []
bobs = []
for _ in range(runs):
    qr = QuantumRegister(key_len, name='qr')
    cr = ClassicalRegister(key_len, name='cr')
    Alice = User(
        name = "Alice",
        key = "",
        basis = [],
        qc = QuantumCircuit(qr, cr, name="a"),
        qr = qr,
        cr = cr,
        n = key_len,
    )
    Bob = User(
        name = "Bob",
        key = "",
        basis = [],
        qc = QuantumCircuit(qr, cr, name="b"),
        qr = qr,
        cr = cr,
        n = key_len,
    )
    alices.append(Alice)
    bobs.append(Bob)

In [11]:
for i in range(runs):
    alices[i].key = int.from_bytes(randbytes(int(key_len/8)))
    # Cast key to binary for encoding
    alices[i].key = np.binary_repr(alices[i].key, key_len) # n is the width
    # Encode key as alice qubits 
    # IBM's qubits are all set to |0> initially
    for index, digit in enumerate(alices[i].key):
        if digit == '1':
            alices[i].qc.x(alices[i].qr[index]) # if key has a '1', change state to |1>
    create_basis(alices[i])
    create_basis(bobs[i])

In [12]:
circs = []
for i in range(runs):
    alice_bob = send_state(alices[i], bobs[i])
    circs.append(transpile(alice_bob,backend))
data = []
with Session(backend = backend) as session:
    sampler = Sampler(session=session)
    job = sampler.run(circs, shots=shots)
    job.result()[i]

for i in range(runs): 
    data.append(job.result()[i].data.cr.get_counts())   


c:\Users\Lowkey\Documents\Code\QuantumExam\.venv\Lib\site-packages\qiskit_ibm_runtime\session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


In [13]:
lens = {}
succ_len = key_len/2 # min key lenght should be half the input qubits
fails = 0
for i in range(runs):
    for key, count in data[i].items():
        bobs[i].key = key[::-1]
        (key_a, key_b) = get_shared_key2(alices[i], bobs[i])
        acc = get_accurracy(key_a, key_b)
        key_len = len(key_a)
        if key_len in lens:
            lens[key_len] += 1 
        else:
            lens[key_len] = 1
        if acc != 1:
            fails +=1*count # counts are a pair of (keys, counts)
total_execs = runs*shots
succ = total_execs - fails
print(f"total executions: {total_execs}")
print(f"Succ rate: {succ/total_execs}")
print(f"Fail rate: {fails/total_execs}")
print(f"Average key length {sum(lens[l] * l for l in lens)/total_execs}")
print(f"Unique lengths: {lens}")

total executions: 20
Succ rate: 0.85
Fail rate: 0.15
Average key length 9.0
Unique lengths: {9: 8, 10: 4, 6: 2, 7: 4, 14: 2}
